In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import numpy as np
import gc
import time
import os

In [3]:
feat_dir = './feat/'
if not os.path.exists(feat_dir):
    os.mkdir(feat_dir)

In [4]:
train_data = pd.read_pickle('./pkl/train_data.pkl')
test_data = pd.read_pickle('./pkl/valid_data.pkl')

In [5]:
pub_info = pd.read_pickle('./pkl/pub_info.pkl')
author_pub_detail = pd.read_pickle('./pkl/author_pub_detail.pkl')

In [6]:
train_data.head(3)

,paper_id,author_name,author_org,author_id,label
0,P9a1gcvg,li_guo,Institute of Pharmacology and Toxicology,8GjtUmBs,0
1,P9a1gcvg,li_guo,Institute of Pharmacology and Toxicology,EShnTfSe,1
2,P9a1gcvg,li_guo,Institute of Pharmacology and Toxicology,t1ruuB9N,0


In [7]:
test_data.head(3)

,paper_id,author_name,author_org,author_id
0,F3Mha4HG,lei_shi,State Key Laboratory of Catalysis,0gL4hj4n
1,F3Mha4HG,lei_shi,State Key Laboratory of Catalysis,0jknXeWe
2,F3Mha4HG,lei_shi,State Key Laboratory of Catalysis,1vE3Drg0


In [8]:
print(train_data.shape)
print(test_data.shape)

(1194996, 5)
(452786, 4)


In [9]:
data = pd.concat([train_data, test_data]).reset_index(drop=True)

In [10]:
data.head()

,author_id,author_name,author_org,label,paper_id
0,8GjtUmBs,li_guo,Institute of Pharmacology and Toxicology,0.0,P9a1gcvg
1,EShnTfSe,li_guo,Institute of Pharmacology and Toxicology,1.0,P9a1gcvg
2,t1ruuB9N,li_guo,Institute of Pharmacology and Toxicology,0.0,P9a1gcvg
3,xLLXKy6I,li_guo,Institute of Pharmacology and Toxicology,0.0,P9a1gcvg
4,jTu2AZES,li_guo,Institute of Pharmacology and Toxicology,0.0,P9a1gcvg


In [11]:
pub_info.columns = ['abstract_a', 'authors_a', 'keywords_a', 'paper_id', 'title_a', 'venue_a', 'year_a', 'orgs_a']
pub_info.head()

,abstract_a,authors_a,keywords_a,paper_id,title_a,venue_a,year_a,orgs_a
0,Extremely long states of bound solitons consis...,"[Ya Liu, Xin Zhao, Jiansheng Liu, Guoqing Hu]","[carbon nanotubes, fibre lasers, laser cavity ...",0009qJgC,Generating ultra-long bound soliton sequences ...,Lasers and Electro-Optics,2014,"[Sch. of Electron. & Inf. Eng., Beihang Univ.,..."
1,,"[Hao He, Jun Yang]","[different part, data graph, variety ofstructu...",000BMUBY,Multiresolution Indexing of XML for Frequent Q...,icde,2004,[]
2,The Yellow River (YR) supplies a large amount ...,"[Weifeng Yang, Min Chen, Guangxue Li, Jianping...","[13C, 15N, East China Sea, Trace element, Yell...",000Fmw78,Relocation of the Yellow River as revealed by ...,Marine Pollution Bulletin,2009,[State Key Laboratory of Marine Environmental ...
3,Our findings suggest that Th17 cells may be in...,"[Xiaowei Wang, Xiaojun Chen, Heng Tang, Jifeng...","[Mycoplasma Pneumoniae, Th17 cells, children, ...",000HsdYy,Increased Frequency of Th17 Cells in Children ...,Journal of clinical laboratory analysis,2016,"[Department of Blood Transfusion, Nanjing Chil..."
4,,"[jun chen, xianran xing, andrew d watson, wei ...",[],000NKgQT,Rapid Synthesis of Multiferroic BiFeO 3 Single...,Chemistry of Materials,2007,"[, , , , , , , , ]"


In [12]:
author_pub_detail.columns = ['author_id', 'paper_ids', 'paper_ids_len', 'abstract_b', 'keywords_b', 'title_b', 'venue_b', 'year_b', 'authors_b', 'orgs_b']
author_pub_detail.head()

,author_id,paper_ids,paper_ids_len,abstract_b,keywords_b,title_b,venue_b,year_b,authors_b,orgs_b
0,004mBKh6,"[TY0MYj83, lRl7m2tU, S6rklGvj, dVAKsHla, 2aTAK...",9,[Robot-assisted single port access surgery (SP...,"[[Manipulators, Surgery, Grippers, Kinematics,...",[A Robotic System With Multichannel Flexible P...,"[IEEE Transactions on Industrial Informatics, ...","[2019, 2018, 2018, 2019, 2019, 2019, 2018, 201...","[[Changsheng Li, Xiaoyi Gu, Xiao Xiao, Chwee M...","[[Department of Biomedical Engineering, Nation..."
1,008b5hIc,[nPuf0kYs],1,[With the development of nuclear power industr...,"[[Ecological security, Groundwater, Radioactiv...",[A review on the study of groundwater and its ...,[Earth Science Frontiers],[2014],"[[Zhanxue Sun, Yuanyuan Liu, Wenjie Ma, Bai Gao]]",[[State Key Laboratory Breeding Base of Nuclea...
2,00BAgHFr,"[TuEkjf5i, kn6wo3ot, 0f2awX7U]",3,[An elevations selection approach of area targ...,"[[EOS application, GA, Optimization], [Feature...",[Elevations selection approach of area target ...,[Geomatics and Information Science of Wuhan Un...,"[2006, 2006, 2007]","[[Jianghan Zhu, Xi Li, Chilong Mao, Lining Zha...","[[, , , ], [, , ], [School of Information Syst..."
3,00M93JDI,"[7IiMiMhZ, rbdaKJow, nbE0DmMh, PJeWna4W, hBHTP...",309,"[ , , , , , , , , , , , , , , , ...","[[nanoparticles, luminescence], [macropore arr...",[Blue Luminescence of ZnO Nanoparticles Based ...,"[advanced functional materials, journal of col...","[2010, 2007, 2007, 1999, 2009, 2003, 2017, 200...","[[Haibo Zeng, Guotao Duan, Yue Li, Shikuan Yan...","[[], [], [], [chinese academy of sciences, chi..."
4,00ShLPmg,"[inFGp3mc, LV5duZL5, jGsCF9Uy, Nii27VXk, uyJus...",18,"[ , , , , , , , , , , , , , , , ...","[[Protein kinase C, Idiopathic thrombocytopeni...",[Determination of protein kinase C activity of...,"[laboratory medicine, journal of clinical hema...","[2006, 2007, 2012, 2011, 2006, 2006, 2009, 200...","[[LIU Fang, WU Changlin, XIAO Hong, CHEN Qun],...","[[.Department of Microbiology and Immunology,G..."


In [13]:
data = data.merge(pub_info, 'left', 'paper_id').merge(author_pub_detail, 'left', 'author_id')

In [14]:
data.shape

(1647782, 21)

In [15]:
data.head(3)

,author_id,author_name,author_org,label,paper_id,abstract_a,authors_a,keywords_a,title_a,venue_a,...,orgs_a,paper_ids,paper_ids_len,abstract_b,keywords_b,title_b,venue_b,year_b,authors_b,orgs_b
0,8GjtUmBs,li_guo,Institute of Pharmacology and Toxicology,0.0,P9a1gcvg,Objective: To establish a simultaneous determi...,"[Fenghe Qiu, Li Liu, Li Guo]","[central nervous drugs, GC-FID, GC-MS, solid-p...",Rapid determination of central nervous drugs i...,Chinese Pharmaceutical Journal,...,"[Institute of Pharmacology and Toxicology, Ins...",[J5KPOi3Y],1,[ ],"[[Beijing, China, Drop trend, Soil Hg]]","[Mercury drop trend in urban soils in Beijing,...",[Journal of Geochemical Exploration],[2013],"[[Hangxin Cheng, Chuandong Zhao, Fei Liu, Ke Y...",[[Key Laboratory of Geochemical Cycling of Car...
1,EShnTfSe,li_guo,Institute of Pharmacology and Toxicology,1.0,P9a1gcvg,Objective: To establish a simultaneous determi...,"[Fenghe Qiu, Li Liu, Li Guo]","[central nervous drugs, GC-FID, GC-MS, solid-p...",Rapid determination of central nervous drugs i...,Chinese Pharmaceutical Journal,...,"[Institute of Pharmacology and Toxicology, Ins...",[P9a1gcvg],1,[Objective: To establish a simultaneous determ...,"[[central nervous drugs, GC-FID, GC-MS, solid-...",[Rapid determination of central nervous drugs ...,[Chinese Pharmaceutical Journal],[1996],"[[Fenghe Qiu, Li Liu, Li Guo]]","[[Institute of Pharmacology and Toxicology, In..."
2,t1ruuB9N,li_guo,Institute of Pharmacology and Toxicology,0.0,P9a1gcvg,Objective: To establish a simultaneous determi...,"[Fenghe Qiu, Li Liu, Li Guo]","[central nervous drugs, GC-FID, GC-MS, solid-p...",Rapid determination of central nervous drugs i...,Chinese Pharmaceutical Journal,...,"[Institute of Pharmacology and Toxicology, Ins...",[T35c9ZAd],1,[Beijing Plain is composed of a suite of consi...,"[[Beijing, Cenozoic, Stratigraphie classificat...",[Core characteristics and preliminary stratigr...,[Geology in China],[2011],"[[Ying-Bo Luan, Xiang-Min Cai, Gao-Xuan Guo, Y...",[[Bureau of Geological Mineral Resources Explo...


### delete paper_id in pos sample

In [16]:
def pidx(p, ps):
    ans = np.nan
    for i, p2 in enumerate(ps):
        if p == p2:
            ans = i
            break
    return ans

In [17]:
%%time
data['idx'] = data.apply(lambda row: pidx(row['paper_id'], row['paper_ids']) if row['label'] == 1 else np.nan, axis=1)

CPU times: user 1min 16s, sys: 1.61 s, total: 1min 18s
Wall time: 1min 18s


In [18]:
from tqdm import tqdm_notebook

cols = ['abstract_b', 'keywords_b', 'title_b', 'venue_b', 'year_b', 'authors_b', 'orgs_b']
for i in tqdm_notebook(range(len(data))):
    if pd.isna(data.loc[i, 'idx']):
        continue
    for c in cols:
        v = list(data.loc[i, c])
        del v[data.loc[i, 'idx'].astype(int)]
        data.set_value(i, c, v)
#         data.loc[i, c] = frozenset(v)

In [19]:
data.to_pickle('./pkl/data.pkl')

In [20]:
print(data.columns)

Index(['author_id', 'author_name', 'author_org', 'label', 'paper_id',
       'abstract_a', 'authors_a', 'keywords_a', 'title_a', 'venue_a', 'year_a',
       'orgs_a', 'paper_ids', 'paper_ids_len', 'abstract_b', 'keywords_b',
       'title_b', 'venue_b', 'year_b', 'authors_b', 'orgs_b', 'idx'],
      dtype='object')


### year

In [21]:
data = pd.read_pickle('./pkl/data.pkl')

In [22]:
data['year_b'] = data['year_b'].apply(lambda x: [0] if len(x) == 0 else x)

In [23]:
data['year_b_min'] = data['year_b'].apply(np.min)
data['year_b_max'] = data['year_b'].apply(np.max)
data['year_b_mean'] = data['year_b'].apply(np.mean)
data['year_b_std'] = data['year_b'].apply(np.std)

In [24]:
data['year_b_mm2'] = (data['year_b_min'] + data['year_b_max']) / 2

In [25]:
for c in ['year_b_min', 'year_b_max', 'year_b_mean', 'year_b_mm2']:
    data[c + '-year_a'] = data[c] - data['year_a']

In [26]:
data['year_inside_range'] = ((data['year_b_min-year_a'] <= 0) & (data['year_b_max-year_a'] >= 0)).astype(int)

In [27]:
cols = ['year_a', 'year_b_min', 'year_b_max', 'year_b_mean', 'year_b_std', 'year_b_mm2',
       'year_b_min-year_a', 'year_b_max-year_a', 'year_b_mean-year_a',
       'year_b_mm2-year_a', 'year_inside_range']

In [28]:
data[cols].to_pickle('./feat/time_feat_a.pkl')

### 'authors', 'orgs'

In [29]:
data = pd.read_pickle('./pkl/data.pkl')
tmp = data[['author_org', 'authors_a', 'orgs_a', 'authors_b', 'orgs_b']]

In [30]:
# import multiprocessing as mp

# def split_df(df, n):
#     chunk_size = int(np.ceil(len(df) / n))
#     return [df[i*chunk_size:(i+1)*chunk_size] for i in range(n)]

# chunk_list = split_df(tmp, 100)

In [31]:
def func(a, b):
    cnt = 0
    for x in b:
        for y in x:
            if a == y:
                cnt += 1
    return cnt

In [32]:
# def process(df):
#     return df.apply(lambda row: func(row['author_org'], row['orgs_b']), axis=1)
 
# with mp.Pool(8) as pool:
#     ret = pool.map(process, chunk_list)
#     print(len(ret))

In [33]:
%%time
tmp['author_org_in_orgs_b_times'] = tmp.apply(lambda row: func(row['author_org'], row['orgs_b']), axis=1)

CPU times: user 4min 11s, sys: 2.64 s, total: 4min 14s
Wall time: 4min 14s


In [34]:
def func(a, b):
    b = set([y for x in b for y in x])
#     b = set([x for x in b])
    a = set(a)
    return len(a & b)

In [35]:
%%time
tmp['author_interset_num'] = tmp.apply(lambda row: func(row['authors_a'], row['authors_b']), axis=1)

CPU times: user 7min 2s, sys: 3.65 s, total: 7min 5s
Wall time: 7min 6s


In [36]:
tmp['author_interset_num/paper_ids_len'] = (tmp['author_interset_num'] / (data['paper_ids_len'] - (data['label'] == 1).astype(int))).fillna(0)

In [37]:
tmp['author_interset_num'].value_counts()

1       1049629
0        397246
2         44245
3         40378
4         35455
5         26070
6         17612
7         11221
8          7733
9          4788
10         2903
11         1800
12         1191
13          680
14          513
15          384
16          279
17          238
20          141
18          138
19          137
21           79
24           63
25           62
22           56
26           48
2409         46
99           46
23           42
52           37
         ...   
1451          1
424           1
973           1
1413          1
439           1
2436          1
2045          1
977           1
3026          1
979           1
981           1
432           1
446           1
1513          1
1482          1
484           1
482           1
480           1
1491          1
1490          1
1487          1
1480          1
447           1
1478          1
1476          1
1475          1
451           1
449           1
448           1
1022          1
Name: author_interset_nu

In [39]:
tmp[['author_org_in_orgs_b_times', 'author_interset_num', 'author_interset_num/paper_ids_len']].to_pickle('./feat/author_org_cross.pkl')

### venue, keywords

In [44]:
# data = pd.read_pickle('./pkl/data.pkl')
tmp = data[['venue_a', 'venue_b', 'keywords_a', 'keywords_b']]
tmp.head()

,venue_a,venue_b,keywords_a,keywords_b
0,Chinese Pharmaceutical Journal,[Journal of Geochemical Exploration],"[central nervous drugs, GC-FID, GC-MS, solid-p...","[[Beijing, China, Drop trend, Soil Hg]]"
1,Chinese Pharmaceutical Journal,[],"[central nervous drugs, GC-FID, GC-MS, solid-p...",[]
2,Chinese Pharmaceutical Journal,[Geology in China],"[central nervous drugs, GC-FID, GC-MS, solid-p...","[[Beijing, Cenozoic, Stratigraphie classificat..."
3,Chinese Pharmaceutical Journal,[Jianzhu Cailiao Xuebao/Journal of Building Ma...,"[central nervous drugs, GC-FID, GC-MS, solid-p...","[[Carbonated concrete, Experimental study, Rea..."
4,Chinese Pharmaceutical Journal,[Chinese Journal of New Drugs],"[central nervous drugs, GC-FID, GC-MS, solid-p...","[[Intramuscular in children, Recombinant human..."


In [59]:
def func(a, b):
    cnt = 0
    for x in b:
        if a.lower() == x.lower():
            cnt += 1
    return cnt

In [60]:
tmp['venue_a_in_venue_b_num'] = tmp.apply(lambda row: func(row['venue_a'], row['venue_b']), axis=1)

In [61]:
def func(a, b):
    bl = []
    for x in b:
        if type(x) == list:
            for y in x:
                if not pd.isna(y):
                    bl.append(y.lower())
    b = set(bl)
    al = []
    if type(a) != float:
        for x in a:
            al.append(x.lower())
    a = set(al)
    return len(a & b)

In [62]:
%%time
tmp['keywords_interset_num'] = tmp.apply(lambda row: func(row['keywords_a'], row['keywords_b']), axis=1)

CPU times: user 9min 43s, sys: 5.54 s, total: 9min 49s
Wall time: 9min 50s


In [63]:
tmp['venue_a_in_venue_b_num/paper_ids_len'] = (tmp['venue_a_in_venue_b_num'] / (data['paper_ids_len'] - (data['label'] == 1).astype(int))).fillna(0)
tmp['keywords_interset_num/paper_ids_len'] = (tmp['keywords_interset_num'] / (data['paper_ids_len'] - (data['label'] == 1).astype(int))).fillna(0)

In [68]:
tmp.head(3)

,venue_a,venue_b,keywords_a,keywords_b,venue_a_in_venue_b_num,keywords_interset_num,venue_a_in_venue_b_num/paper_ids_len,keywords_interset_num/paper_ids_len
0,Chinese Pharmaceutical Journal,[Journal of Geochemical Exploration],"[central nervous drugs, GC-FID, GC-MS, solid-p...","[[Beijing, China, Drop trend, Soil Hg]]",0,0,0.0,0.0
1,Chinese Pharmaceutical Journal,[],"[central nervous drugs, GC-FID, GC-MS, solid-p...",[],0,0,0.0,0.0
2,Chinese Pharmaceutical Journal,[Geology in China],"[central nervous drugs, GC-FID, GC-MS, solid-p...","[[Beijing, Cenozoic, Stratigraphie classificat...",0,0,0.0,0.0


In [69]:
tmp[['venue_a_in_venue_b_num', 'keywords_interset_num', 'venue_a_in_venue_b_num/paper_ids_len', 'keywords_interset_num/paper_ids_len']].to_pickle('./feat/keyword_venue_cross.pkl')

In [72]:
data.head()

,author_id,author_name,author_org,label,paper_id,abstract_a,authors_a,keywords_a,title_a,venue_a,...,paper_ids,paper_ids_len,abstract_b,keywords_b,title_b,venue_b,year_b,authors_b,orgs_b,idx
0,8GjtUmBs,li_guo,Institute of Pharmacology and Toxicology,0.0,P9a1gcvg,Objective: To establish a simultaneous determi...,"[Fenghe Qiu, Li Liu, Li Guo]","[central nervous drugs, GC-FID, GC-MS, solid-p...",Rapid determination of central nervous drugs i...,Chinese Pharmaceutical Journal,...,[J5KPOi3Y],1,[ ],"[[Beijing, China, Drop trend, Soil Hg]]","[Mercury drop trend in urban soils in Beijing,...",[Journal of Geochemical Exploration],[2013],"[[Hangxin Cheng, Chuandong Zhao, Fei Liu, Ke Y...",[[Key Laboratory of Geochemical Cycling of Car...,NaN
1,EShnTfSe,li_guo,Institute of Pharmacology and Toxicology,1.0,P9a1gcvg,Objective: To establish a simultaneous determi...,"[Fenghe Qiu, Li Liu, Li Guo]","[central nervous drugs, GC-FID, GC-MS, solid-p...",Rapid determination of central nervous drugs i...,Chinese Pharmaceutical Journal,...,[P9a1gcvg],1,[],[],[],[],[],[],[],0.0
2,t1ruuB9N,li_guo,Institute of Pharmacology and Toxicology,0.0,P9a1gcvg,Objective: To establish a simultaneous determi...,"[Fenghe Qiu, Li Liu, Li Guo]","[central nervous drugs, GC-FID, GC-MS, solid-p...",Rapid determination of central nervous drugs i...,Chinese Pharmaceutical Journal,...,[T35c9ZAd],1,[Beijing Plain is composed of a suite of consi...,"[[Beijing, Cenozoic, Stratigraphie classificat...",[Core characteristics and preliminary stratigr...,[Geology in China],[2011],"[[Ying-Bo Luan, Xiang-Min Cai, Gao-Xuan Guo, Y...",[[Bureau of Geological Mineral Resources Explo...,NaN
3,xLLXKy6I,li_guo,Institute of Pharmacology and Toxicology,0.0,P9a1gcvg,Objective: To establish a simultaneous determi...,"[Fenghe Qiu, Li Liu, Li Guo]","[central nervous drugs, GC-FID, GC-MS, solid-p...",Rapid determination of central nervous drugs i...,Chinese Pharmaceutical Journal,...,[OQ7F7UAh],1,[According to the experiment of realkalization...,"[[Carbonated concrete, Experimental study, Rea...",[Influencing factor of realkalization techniqu...,[Jianzhu Cailiao Xuebao/Journal of Building Ma...,[2008],"[[Wen-Jun Qu, Yan Xiong, Li Guo]]","[[School of Civil Engineering, School of Civil...",NaN
4,jTu2AZES,li_guo,Institute of Pharmacology and Toxicology,0.0,P9a1gcvg,Objective: To establish a simultaneous determi...,"[Fenghe Qiu, Li Liu, Li Guo]","[central nervous drugs, GC-FID, GC-MS, solid-p...",Rapid determination of central nervous drugs i...,Chinese Pharmaceutical Journal,...,[NRR6kM3z],1,[Objective: To evaluate the tolerance and safe...,"[[Intramuscular in children, Recombinant human...",[Tolerance and safety of recombinant human int...,[Chinese Journal of New Drugs],[2011],"[[Wei Sun, Yi Qiao, Li Guo, Yu-Hong Cao, Lin Y...","[[Department of Pharmacy, Department of Pharma...",NaN


### len

In [3]:
# data = pd.read_pickle('./pkl/data.pkl')

In [6]:
# data['paper_num'] = data['paper_ids_len'] - data['label']

In [9]:
# data[['paper_num']].to_pickle('./feat/paper_num.pkl')